In [53]:
require "hdf5"
require "optim"
require "nn"

In [67]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]


--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()

--MEMM
X_train_MEMM = f:read("X_train_MEMM"):all()
X_valid_MEMM = f:read("X_valid_MEMM"):all()
X_valid_sen_MEMM = f:read("X_valid_sen_MEMM"):all()
nfeaturesMEMM = f:read("nfeaturesMEMM"):all()[1]

C = nclasses

In [55]:
function process(inputs, numwords)
    processed = inputs:clone()
    for i = 1, inputs:size(1) do
        processed[i][2] = inputs[i][2] + numwords
    end
    return processed
end

function feval(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_:size(1)):sub(1,bsize)

    local x = torch.Tensor(bsize, X_:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    local y_ = torch.Tensor(bsize, 1)

    
    for i=1,bsize do
        x[i] = X_[idx[i]]
        
        y_[i] = y[idx[i]]
    end
    
    y_ = y_:squeeze()
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    
    return loss_w, dl_dw
            
end

function predict(input, index, network)
    logscore = torch.log(nn.SoftMax():forward(h:forward(input:sub(index,index))))
    return logscore:expand(C,C)
end

function viterbi(observations, logscore)
    local initial = torch.zeros(nclasses,1) + .00001
    initial[8] = 1.0
    initial = initial / torch.sum(initial)
    
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = torch.log(initial):max(2)--(init+ emi[observations[1]]):max(2)
    max_table[1] = maxes
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
        pred = predict(observations, i, logscore)
        scores = pred + maxes:view(1, C):expand(C, C)
        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)
        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
        end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end
    
    return classes:sub(1,-1)
end

function predict_tags(X)
    local predictions = torch.zeros(X:size(1), X:size(2))
    for i=1,X:size(1) do
        local sen = (X[i]:sub(1,torch.nonzero(X[{{i,i},{},{1,1}}]):size(1))):squeeze()
   
        local p = viterbi(sen, predict)
        
        predictions[{{i,i},{1,p:size(1)}}] = p
    end
    return predictions
end

In [56]:
for i = 1,X_valid_sen_MEMM:size(1) do
    X_valid_sen_MEMM[i] = process(X_valid_sen_MEMM[i], nwords)
end
    
X_ = process(X_train_MEMM, nwords)
y = Y_train

In [ ]:
addB = nn.Add(nclasses)
lookup = nn.LookupTable(nwords+nclasses,nclasses)
sum = nn.Sum(2)
softmax = nn.LogSoftMax()
h = nn.Sequential()
h:add(lookup)
h:add(sum)
h:add(addB)
mse = nn.CrossEntropyCriterion()

w, dl_dw = h:getParameters()

In [ ]:
batch = 1000

-- cycle on data
for i = 1,5000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 100 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

In [65]:
a =X_test_sen_MEMM
prediction = viterbi(a[1]:sub(1,12), h)
predictions = predict_tags(a)

[string "a =X_test_sen_MEMM..."]:2: attempt to index global 'a' (a nil value)
stack traceback:
	[string "a =X_test_sen_MEMM..."]:2: in main chunk
	[C]: in function 'xpcall'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/Vincent/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010b7bbbd0: 

In [63]:
-- Writes to file in Kaggle prediction format
-- p : prediction tensor
-- fname : output filename
function kaggle_format(p, fname)
    local f = io.open("predictions/" .. fname, "w")
    f:write("ID,Labels\n")
    for i=1,p:size(1) do
        local s = ""
        local t = {[1]="O",[2]="PER",[3]="LOC",[4]="ORG",[5]="MISC",[6]="MISC",[7]="LOC",[8]="O",[9]="O"}
        local prev_tag = 0
        local prev_idx = 0
        for j=2,p:size(2) do
            if p[i][j] == 0 then
                break
            end
            if p[i][j] ~= 1 then
                if prev_tag == 0 then
                    s = s .. t[p[i][j]] .. "-" .. tostring(j-1)
                elseif p[i][j] == prev_tag and prev_idx == j-1 then
                    s = s .. "-" .. tostring(j-1) 
                else
                    s = s .. " " .. t[p[i][j]] .. "-" .. tostring(j-1)
                end
                prev_tag = p[i][j]
                prev_idx = j
            end
        end
        f:write(i .. "," .. s .. "\n")
    end
    f:close()
end

In [64]:
kaggle_format(predictions, "pred_" .. tostring(os.time()) .. "_hmm.csv")